<a href="https://colab.research.google.com/github/jkchandalia/nlp/blob/main/Poetry_w_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install torch


In [3]:
import torch
from datasets import load_dataset

In [4]:
train_file = "../data/labeled_aggregate/labeled_reddit_vs_gpt_haikus_199_train.csv"
test_file = "../data/labeled_aggregate/labeled_reddit_vs_gpt_haikus_199_test.csv"

dataset = load_dataset("csv", data_files={"train": [train_file], "test": [test_file]})

Found cached dataset csv (/Users/danaengebretson/.cache/huggingface/datasets/csv/default-6603b4cd03e98688/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset['train']['text'][:10]

["Silent vigilance, \n A cat's tranquil guard duty, \n Ever watchful eyes.",
 'Why is it so hard? \n Just to get a job right now. \n Paying living wage.',
 'An ancient city. \n Silent sands consume his name. \n Ozymandias.',
 'Clear was your anger. \n Strange how I prayed for you. \n Find the love you seek.',
 "People's dilemma. \n In the last breath of freedom. \n Thunderous applause.",
 'Graceful and agile, \n A hunter prowls through the night, \n Silent and precise.',
 'I look for shelter. \n And find a new hiding place. \n Under the pines.',
 'Progress is all gone. \n I gave into my vices. \n Time to start over.',
 'A peaceful sunrise. \n Discovers the idle streets. \n The drunkard blinded.',
 'Nimble as they play, \n Springy and agile they leap, \n Kittens at their best.']

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/159 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [7]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [9]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, label_str. If text, label_str are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/danaengebretson/opt/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 159
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 60
  Number of trainable parameters = 66955010


  0%|          | 0/60 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, label_str. If text, label_str are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 8


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.047660477459430695, 'eval_accuracy': 1.0, 'eval_runtime': 11.9256, 'eval_samples_per_second': 3.354, 'eval_steps_per_second': 0.419, 'epoch': 1.0}


...

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, label_str. If text, label_str are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 8


  0%|          | 0/5 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.0055450135841965675, 'eval_accuracy': 1.0, 'eval_runtime': 11.8147, 'eval_samples_per_second': 3.386, 'eval_steps_per_second': 0.423, 'epoch': 3.0}
{'train_runtime': 679.0468, 'train_samples_per_second': 0.702, 'train_steps_per_second': 0.088, 'train_loss': 0.13616747856140138, 'epoch': 3.0}


TrainOutput(global_step=60, training_loss=0.13616747856140138, metrics={'train_runtime': 679.0468, 'train_samples_per_second': 0.702, 'train_steps_per_second': 0.088, 'train_loss': 0.13616747856140138, 'epoch': 3.0})

In [16]:
model.to('cpu')

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [17]:

inputs = tokenizer("Majestic felines, \n Prowling through the night with grace, \n Purring by the fire.", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()


In [18]:
predicted_class_id

0

In [20]:
inputs = tokenizer("Orange tubby tummy, \n he begs me for belly rubs, \n by biting my feet.", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()

predicted_class_id

0

In [22]:
inputs = tokenizer("May all improvements, \n be shouted off rooftops, \n so the ideas spread.", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()

predicted_class_id

1